In [ ]:
'''   

input - min, max, average temp from day before + min, max, average humidity from day before

https://www.wunderground.com/history/daily/us/tx/austin/KAUS/date/2024-3-15

https://www.wunderground.com/history/daily/us/il/chicago/KMDW/date/2024-3-15

https://www.wunderground.com/history/daily/us/fl/miami/KMIA/date/2024-3-15


'''



In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import datetime
import time

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Function to format the date string
def format_date(year, month):
    return f"{year}-{month:02}"

# Define the range of months to loop through
start_year = 2021
end_year = 2024
months = [2, 3, 4]  # February, March, April

# Create the folder if it doesn't exist
folder_name = "mia_daily_average_febmarchapril"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Loop through each year and month
for year in range(start_year, end_year + 1):
    for month in months:
        try:
            start_time = time.time()  # Record the start time
            year_month = format_date(year, month)
            url = f'https://www.wunderground.com/history/monthly/us/fl/miami/KMIA/date/{year_month}'
            driver.get(url)

            # Add implicit wait with a timeout of 10 seconds
            driver.implicitly_wait(10)

            # Locate the table element using WebDriverWait for better reliability
            table_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]")))

            # Extract the text from the table
            table_text = table_element.text

            # Save table_text to a text file inside the folder
            filename = os.path.join(folder_name, f"mia_{year_month}.txt")
            with open(filename, "w") as file:
                file.write(table_text)
            
            end_time = time.time()  # Record the end time
            elapsed_time = end_time - start_time  # Calculate elapsed time
            print(f"Saved table for {year_month} to {filename}. Elapsed time: {elapsed_time:.2f} seconds")
        except Exception as e:
            print(f"Error occurred while processing {year_month}: {e}")
            continue
        
        # Check if elapsed time is greater than 3 minutes, move to the next month
        if elapsed_time > 180:
            print(f"Elapsed time exceeded 3 minutes for {year_month}. Moving to the next month.")
            break

driver.quit()


Saved table for 2021-02 to mia_daily_average_febmarchapril/mia_2021-02.txt. Elapsed time: 112.56 seconds
Saved table for 2021-03 to mia_daily_average_febmarchapril/mia_2021-03.txt. Elapsed time: 66.27 seconds
Saved table for 2021-04 to mia_daily_average_febmarchapril/mia_2021-04.txt. Elapsed time: 26.08 seconds
Saved table for 2022-02 to mia_daily_average_febmarchapril/mia_2022-02.txt. Elapsed time: 29.11 seconds
Saved table for 2022-03 to mia_daily_average_febmarchapril/mia_2022-03.txt. Elapsed time: 24.25 seconds
Saved table for 2022-04 to mia_daily_average_febmarchapril/mia_2022-04.txt. Elapsed time: 33.64 seconds
Saved table for 2023-02 to mia_daily_average_febmarchapril/mia_2023-02.txt. Elapsed time: 34.11 seconds
Saved table for 2023-03 to mia_daily_average_febmarchapril/mia_2023-03.txt. Elapsed time: 33.82 seconds
Saved table for 2023-04 to mia_daily_average_febmarchapril/mia_2023-04.txt. Elapsed time: 31.82 seconds
Saved table for 2024-02 to mia_daily_average_febmarchapril/mia_

In [8]:
import pandas as pd
from io import StringIO

# Read the text file
with open('chi_daily_average_febmarchapril/chi_2020-02.txt', 'r') as file:
    data = file.read()

# Replace spaces with commas
data = data.replace(' ', ',')

with open('weather_data.csv', 'w') as file:
    file.write(data)

In [10]:
# read in csv
# if line two is feb, numday is 28, if march numday is 31, april 30
# delete up to line numday+3
# save next numday lines in format (temp_max, temp_avg, temp_min)
# delete next line
# save next numday lines in format (Dew_Point_max, Dew_Point_avg, Dew_Point_min)
# delete next line
# same for humidity
#delete next line
# same for windspeed
# dlete next line
# same for pressure
# each day  should have max, min, avg for each of the various feilds
# end dataframe should be 15 columns with numday rows



In [102]:
import pandas as pd

# Read in the CSV file
df = pd.read_csv('weather_data.csv')

# Determine the number of days in the current month
#ADD CHECK FOR LEAP YEAR
second_line = df.iloc[0, 0]
if second_line == 'Feb':
    num_days = 28
elif second_line == 'Mar':
    num_days = 31
elif second_line == 'Apr':
    num_days = 30
else:
    raise ValueError("Unsupported month")

df = df.iloc[:, :3]
df = df.iloc[num_days + 3:, :]

# Temperature DataFrame
temp_df = df.iloc[0:num_days + 1, 0:4].reset_index(drop=True)
temp_df.columns = ['Max_Temp', 'Avg_Temp', 'Min_Temp']

# Dew Point DataFrame
dew_df = df.iloc[num_days + 2:2*num_days + 3, 0:4].reset_index(drop=True)
dew_df.columns = ['Max_Dew', 'Avg_Dew', 'Min_Dew']

# Humidity DataFrame
humidity_df = df.iloc[2*num_days + 4:3*num_days + 5, 0:4].reset_index(drop=True)
humidity_df.columns = ['Max_Humidity', 'Avg_Humidity', 'Min_Humidity']

# Wind Speed DataFrame
wind_df = df.iloc[3*num_days + 6:4*num_days + 7, 0:4].reset_index(drop=True)
wind_df.columns = ['Max_Wind', 'Avg_Wind', 'Min_Wind']

# Pressure DataFrame
pressure_df = df.iloc[4*num_days + 8:5*num_days + 9, 0:4].reset_index(drop=True)
pressure_df.columns = ['Max_Pressure', 'Avg_Pressure', 'Min_Pressure']

combined_df = pd.concat([temp_df, dew_df, humidity_df, wind_df, pressure_df], axis=1)
combined_df['Date'] = pd.date_range(start=f"2020-'Feb'-01", periods=len(combined_df), freq='D')

combined_df.head(29)




,Max_Temp,Avg_Temp,Min_Temp,Max_Dew,Avg_Dew,Min_Dew,Max_Humidity,Avg_Humidity,Min_Humidity,Max_Wind,Avg_Wind,Min_Wind,Max_Pressure,Avg_Pressure,Min_Pressure,Date
0,38,34.6,33,34,30.9,28,96,86.4,76,20,11.5,6,29.4,29.2,29.0,2020-02-01
1,54,43.3,35,34,31.8,29,82,64.8,45,23,15.0,3,29.1,28.9,28.8,2020-02-02
2,46,38.8,31,35,31.8,28,89,76.5,63,17,8.9,0,29.1,29.1,29.0,2020-02-03
3,38,34.8,31,32,25.6,19,79,69.4,61,28,20.6,13,29.5,29.3,29.1,2020-02-04
4,31,28.8,27,25,21.0,14,89,73.8,50,17,13.1,8,29.5,29.3,29.1,2020-02-05
5,31,29.0,28,26,23.9,20,89,81.5,69,21,12.0,6,29.1,29.0,28.9,2020-02-06
6,37,29.5,22,23,16.8,11,84,61.7,34,14,8.6,5,29.3,29.1,28.9,2020-02-07
7,35,30.4,27,26,21.7,18,89,70.8,52,12,7.4,5,29.6,29.4,29.3,2020-02-08
8,37,32.2,23,34,27.8,18,93,84.5,70,18,11.2,6,29.6,29.4,29.2,2020-02-09
9,36,30.6,27,29,24.6,21,88,78.7,64,14,7.8,0,29.5,29.4,29.3,2020-02-10


In [120]:
import pandas as pd
import os

# Function to convert text files to CSV with space as delimiter
def txt_to_csv(file_path):
    csv_file_path = file_path.replace('.txt', '.csv')
    with open(file_path, 'r') as txt_file:
        lines = txt_file.readlines()
        with open(csv_file_path, 'w') as csv_file:
            for line in lines:
                csv_file.write(','.join(line.split()) + '\n')

# Specify the folder path
folder_path = 'chi_daily_average_febmarchapril'

# Initialize an empty list to store all DataFrames
all_dataframes = []

# Determine the number of days in March 2024
current_year = 2024
current_month = 3
num_days_march_2024 = 18

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    # Convert text file to CSV
    if file_name.endswith('.txt'):
        txt_file_path = os.path.join(folder_path, file_name)
        txt_to_csv(txt_file_path)
        file_name = file_name.replace('.txt', '.csv')
    
    # Extract month information from the file name
    file_name_parts = file_name.split('_')[1].split('-')
    year, month = file_name_parts[0], file_name_parts[1].replace('.csv', '')
    month_abbr = pd.to_datetime(month, format='%m').strftime('%b')
    
    # Determine the number of days in the current month
    if month == '02':
        num_days = 28 if not (int(year) % 4 == 0 and (int(year) % 100 != 0 or int(year) % 400 == 0)) else 29  # Leap year check
    elif month in ['04', '06', '09', '11']:
        num_days = 30
    elif year == str(current_year) and month == str(current_month):
        num_days = num_days_march_2024
    else:
        num_days = 31

    # Read the CSV file into a DataFrame
    df = pd.read_csv(os.path.join(folder_path, file_name))
    
    df = df.iloc[:, :3]
    df = df.iloc[num_days + 3:, :]
    
    # Skip lines containing "Min", "Max", "Avg"
    df = df[~df.iloc[:, 0].str.contains('Min|Max|Avg|Total')]
    
    # Temperature DataFrame
    temp_df = df.iloc[0:num_days, 0:3].reset_index(drop=True)
    temp_df.columns = ['Max_Temp', 'Avg_Temp', 'Min_Temp']

    # Dew Point DataFrame
    dew_df = df.iloc[num_days:2*num_days, 0:3].reset_index(drop=True)
    dew_df.columns = ['Max_Dew', 'Avg_Dew', 'Min_Dew']

    # Humidity DataFrame
    humidity_df = df.iloc[2*num_days:3*num_days, 0:3].reset_index(drop=True)
    humidity_df.columns = ['Max_Humidity', 'Avg_Humidity', 'Min_Humidity']

    # Wind Speed DataFrame
    wind_df = df.iloc[3*num_days:4*num_days, 0:3].reset_index(drop=True)
    wind_df.columns = ['Max_Wind', 'Avg_Wind', 'Min_Wind']

    # Pressure DataFrame
    pressure_df = df.iloc[4*num_days:5*num_days, 0:3].reset_index(drop=True)
    pressure_df.columns = ['Max_Pressure', 'Avg_Pressure', 'Min_Pressure']
    
    # Combine the data for each feature into one DataFrame
    file_combined_df = pd.concat([temp_df, dew_df, humidity_df, wind_df, pressure_df], axis=1)
    
    # Add a date column
    file_combined_df['Date'] = pd.date_range(start=f"{year}-{month_abbr}-01", periods=len(file_combined_df), freq='D')
    
    # Append the combined DataFrame to the list
    all_dataframes.append(file_combined_df)

# Concatenate all DataFrames in the list to create one big DataFrame
big_dataframe = pd.concat(all_dataframes, ignore_index=True)

# Save the combined DataFrame to a CSV file
big_dataframe.to_csv('combined_data.csv', index=False)


